<p float="center">
  <img src="images/horizontal.png" alt="Coiled logo" width="415" hspace="10"/>
  <img src="images/dask_horizontal_no_pad.svg" alt="Dask logo" width="400" hspace="10" />
</p>

# Scalable Data Science

In this notebook, we'll 

* Perform a basic analytics and ETL workflow on the NYC taxi dataset using Pandas;
* Scale up this workflow to a dataset that doesn't fit in RAM using Dask;
* (Optional) Scale out this workflow to leverage a cluster on the Cloud using Coiled.

The workflow is intentionally boring so that we can see the power of scalable data science immediately: we'll load some data, convert some data types for more efficient storage, add a new column, save the new DataFrame to file.

In the notebooks that follow, we'll jump into more interesting examples, including machine learning.

*A bit about me:* I'm Hugo Bowne-Anderson, Head of Data Science Evangelism and Marketing at [Coiled](coiled.io/). We build products that bring the power of scalable data science and machine learning to you, such as single-click hosted clusters on the cloud. We want to take the DevOps out of data science so you can get back to your real job. If you're interested in taking Coiled for a test drive, you can sign up for our [free Beta here](beta.coiled.io/).

Before scaling up, let's look at a common workflow in Pandas.

## 1. Pandas: Convert CSV to Parquet and Engineer a Feature

<img src="images/pandas-logo.svg" alt="pandas logo" style="width: 500px;"/>

In the following, we'll 

* use Pandas to load in part of the NYC taxi dataset from a CSV, 
* massage the data, 
* engineer a feature, 
* compute the average tip as a function of the number of passengers, and 
* save to [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) (far more efficient than CSV, but not human-readable).

If you're following along in Binder, you won't be able to execute the code but you can read it.

### Download the data from Amazon

In [ ]:
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-{01..12}.csv

Note: this will take at least several minutes to download the above.

### Investigate data locally with Pandas


In [1]:
# Import pandas and read in beginning of 1st file
import pandas as pd
df = pd.read_csv(
    "data_taxi/yellow_tripdata_2019-01.csv", 
    nrows=10000,
)
df

VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0            1  2019-01-01 00:46:40   2019-01-01 00:53:20                1   
1            1  2019-01-01 00:59:47   2019-01-01 01:18:59                1   
2            2  2018-12-21 13:48:30   2018-12-21 13:52:40                3   
3            2  2018-11-28 15:52:25   2018-11-28 15:55:45                5   
4            2  2018-11-28 15:56:57   2018-11-28 15:58:33                5   
...        ...                  ...                   ...              ...   
9995         2  2019-01-01 00:20:18   2019-01-01 00:24:40                2   
9996         2  2019-01-01 00:25:29   2019-01-01 00:45:00                2   
9997         2  2019-01-01 00:47:19   2019-01-01 00:55:38                2   
9998         2  2019-01-01 00:31:41   2019-01-01 00:56:26                1   
9999         2  2019-01-01 00:58:04   2019-01-01 01:15:24                1   

      trip_distance  RatecodeID store_and_fwd_flag  PULocationID  \
0              1.50           1                  N           151   
1              2.60           1                  N           239   
2              0.00           1                  N           236   
3              0.00           1                  N           193   
4              0.00           2                  N           193   
...             ...         ...                ...           ...   
9995           1.03           1                  N           263   
9996           4.97           1                  N           141   
9997           2.16           1                  N           231   
9998           1.08           1                  N           249   
9999           2.26           1                  N            68   

      DOLocationID  payment_type  fare_amount  extra  mta_tax  tip_amount  \
0              239             1          7.0    0.5      0.5        1.65   
1              246             1         14.0    0.5      0.5        1.00   
2              236             1          4.5    0.5      0.5        0.00   
3              193             2          3.5    0.5      0.5        0.00   
4              193             2         52.0    0.0      0.5        0.00   
...            ...           ...          ...    ...      ...         ...   
9995           141             2          5.5    0.5      0.5        0.00   
9996           231             1         17.0    0.5      0.5        3.66   
9997            33             1          9.0    0.5      0.5        2.06   
9998           186             1         15.0    0.5      0.5        1.00   
9999           162             2         12.5    0.5      0.5        0.00   

      tolls_amount  improvement_surcharge  total_amount  congestion_surcharge  
0              0.0                    0.3          9.95                   NaN  
1              0.0                    0.3         16.30                   NaN  
2              0.0                    0.3          5.80                   NaN  
3              0.0                    0.3          7.55                   NaN  
4              0.0                    0.3         55.55                   NaN  
...            ...                    ...           ...                   ...  
9995           0.0                    0.3          6.80                   NaN  
9996           0.0                    0.3         21.96                   NaN  
9997           0.0                    0.3         12.36                   NaN  
9998           0.0                    0.3         17.30                   NaN  
9999           0.0                    0.3         13.80                   NaN  

[10000 rows x 18 columns]

In [3]:
df.dtypes

VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
RatecodeID                 int64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

In [ ]:
# Check out head of file
!head data_taxi/yellow_tripdata_2019-01.csv

### Massage data types 

Before we convert to parquet format, let's clean up our types a little

In [4]:
%%time

# Read in data, parse dates
df = pd.read_csv(
    "data_taxi/yellow_tripdata_2019-01.csv", 
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
)
df

CPU times: user 20.1 s, sys: 4.74 s, total: 24.8 s
Wall time: 26 s


VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0               1  2019-01-01 00:46:40   2019-01-01 00:53:20                1   
1               1  2019-01-01 00:59:47   2019-01-01 01:18:59                1   
2               2  2018-12-21 13:48:30   2018-12-21 13:52:40                3   
3               2  2018-11-28 15:52:25   2018-11-28 15:55:45                5   
4               2  2018-11-28 15:56:57   2018-11-28 15:58:33                5   
...           ...                  ...                   ...              ...   
7667787         2  2019-01-31 23:57:36   2019-02-01 00:18:39                1   
7667788         2  2019-01-31 23:32:03   2019-01-31 23:33:11                1   
7667789         2  2019-01-31 23:36:36   2019-01-31 23:36:40                1   
7667790         2  2019-01-31 23:14:53   2019-01-31 23:15:20                1   
7667791         2  2019-01-31 23:12:49   2019-01-31 23:14:08                1   

         trip_distance  RatecodeID store_and_fwd_flag  PULocationID  \
0                 1.50           1                  N           151   
1                 2.60           1                  N           239   
2                 0.00           1                  N           236   
3                 0.00           1                  N           193   
4                 0.00           2                  N           193   
...                ...         ...                ...           ...   
7667787           4.79           1                  N           263   
7667788           0.00           1                  N           193   
7667789           0.00           1                  N           264   
7667790           0.00           1                  N           264   
7667791           0.00           1                  N           193   

         DOLocationID  payment_type  fare_amount  extra  mta_tax  tip_amount  \
0                 239             1          7.0    0.5      0.5        1.65   
1                 246             1         14.0    0.5      0.5        1.00   
2                 236             1          4.5    0.5      0.5        0.00   
3                 193             2          3.5    0.5      0.5        0.00   
4                 193             2         52.0    0.0      0.5        0.00   
...               ...           ...          ...    ...      ...         ...   
7667787             4             1         18.0    0.5      0.5        3.86   
7667788           193             1          0.0    0.0      0.0        0.00   
7667789           264             1          0.0    0.0      0.0        0.00   
7667790             7             1          0.0    0.0      0.0        0.00   
7667791           193             1          0.0    0.0      0.0        0.00   

         tolls_amount  improvement_surcharge  total_amount  \
0                 0.0                    0.3          9.95   
1                 0.0                    0.3         16.30   
2                 0.0                    0.3          5.80   
3                 0.0                    0.3          7.55   
4                 0.0                    0.3         55.55   
...               ...                    ...           ...   
7667787           0.0                    0.3         23.16   
7667788           0.0                    0.0          0.00   
7667789           0.0                    0.0          0.00   
7667790           0.0                    0.0          0.00   
7667791           0.0                    0.0          0.00   

         congestion_surcharge  
0                         NaN  
1                         NaN  
2                         NaN  
3                         NaN  
4                         NaN  
...                       ...  
7667787                   0.0  
7667788                   0.0  
7667789                   0.0  
7667790                   0.0  
7667791                   0.0  

[7667792 rows x 18 columns]

In [5]:
%%time

# Alter data types for efficiency
df = df.astype({
    "VendorID": "uint8",
    "passenger_count": "uint8",
    "RatecodeID": "uint8",
    "store_and_fwd_flag": "category",
    "PULocationID": "uint16",
    "DOLocationID": "uint16",    
})

# Create new feature in dataset: tip_ratio
df["tip_ratio"] = df.tip_amount / df.total_amount

CPU times: user 1.39 s, sys: 1.24 s, total: 2.63 s
Wall time: 2.86 s


## Basic Analytics

In [6]:
%%time

# Compute average tip as a function of the number of passengers
df.groupby("passenger_count").tip_amount.mean()

CPU times: user 116 ms, sys: 22.7 ms, total: 139 ms
Wall time: 140 ms


passenger_count
0    1.786901
1    1.828308
2    1.833877
3    1.795579
4    1.702710
5    1.869868
6    1.856830
7    6.542632
8    6.480690
9    3.116667
Name: tip_amount, dtype: float64

### Convert to Parquet

In [7]:
%%time
df.to_parquet("data_taxi/yellow_tripdata_2019-01.parq")

CPU times: user 5.87 s, sys: 544 ms, total: 6.41 s
Wall time: 5.75 s


In [8]:
%%time

# Read from parquet and time it
df = pd.read_parquet("data_taxi/yellow_tripdata_2019-01.parq")
df

CPU times: user 3.15 s, sys: 3.08 s, total: 6.23 s
Wall time: 4.68 s


VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0               1  2019-01-01 00:46:40   2019-01-01 00:53:20                1   
1               1  2019-01-01 00:59:47   2019-01-01 01:18:59                1   
2               2  2018-12-21 13:48:30   2018-12-21 13:52:40                3   
3               2  2018-11-28 15:52:25   2018-11-28 15:55:45                5   
4               2  2018-11-28 15:56:57   2018-11-28 15:58:33                5   
...           ...                  ...                   ...              ...   
7667787         2  2019-01-31 23:57:36   2019-02-01 00:18:39                1   
7667788         2  2019-01-31 23:32:03   2019-01-31 23:33:11                1   
7667789         2  2019-01-31 23:36:36   2019-01-31 23:36:40                1   
7667790         2  2019-01-31 23:14:53   2019-01-31 23:15:20                1   
7667791         2  2019-01-31 23:12:49   2019-01-31 23:14:08                1   

         trip_distance  RatecodeID store_and_fwd_flag  PULocationID  \
0                 1.50           1                  N           151   
1                 2.60           1                  N           239   
2                 0.00           1                  N           236   
3                 0.00           1                  N           193   
4                 0.00           2                  N           193   
...                ...         ...                ...           ...   
7667787           4.79           1                  N           263   
7667788           0.00           1                  N           193   
7667789           0.00           1                  N           264   
7667790           0.00           1                  N           264   
7667791           0.00           1                  N           193   

         DOLocationID  payment_type  fare_amount  extra  mta_tax  tip_amount  \
0                 239             1          7.0    0.5      0.5        1.65   
1                 246             1         14.0    0.5      0.5        1.00   
2                 236             1          4.5    0.5      0.5        0.00   
3                 193             2          3.5    0.5      0.5        0.00   
4                 193             2         52.0    0.0      0.5        0.00   
...               ...           ...          ...    ...      ...         ...   
7667787             4             1         18.0    0.5      0.5        3.86   
7667788           193             1          0.0    0.0      0.0        0.00   
7667789           264             1          0.0    0.0      0.0        0.00   
7667790             7             1          0.0    0.0      0.0        0.00   
7667791           193             1          0.0    0.0      0.0        0.00   

         tolls_amount  improvement_surcharge  total_amount  \
0                 0.0                    0.3          9.95   
1                 0.0                    0.3         16.30   
2                 0.0                    0.3          5.80   
3                 0.0                    0.3          7.55   
4                 0.0                    0.3         55.55   
...               ...                    ...           ...   
7667787           0.0                    0.3         23.16   
7667788           0.0                    0.0          0.00   
7667789           0.0                    0.0          0.00   
7667790           0.0                    0.0          0.00   
7667791           0.0                    0.0          0.00   

         congestion_surcharge  tip_ratio  
0                         NaN   0.165829  
1                         NaN   0.061350  
2                         NaN   0.000000  
3                         NaN   0.000000  
4                         NaN   0.000000  
...                       ...        ...  
7667787                   0.0   0.166667  
7667788                   0.0        NaN  
7667789                   0.0        NaN  
7667790                   0.0        NaN  
7667

**Recap:** We have

* used Pandas to load in part of the NYC taxi dataset from a CSV, 
* massaged the data, 
* engineered a feature, 
* computed the average tip as a function of the number of passengers, and 
* saved to [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) (far more efficient than CSV, but not human-readable).

### Operate on many files in a for loop?

We could do this, but it's unpleasant

```python
for filename in glob("~/data/nyctaxi/yellow_tripdata_2019-*.parq"):
    df = pd.read_csv(filename)
    ...
    df.to_parquet(...)
```

## 2. Use Dask locally to process the full dataset

<img src="images/dask_horizontal_no_pad.svg" alt="Dask logo" style="width: 500px;"/>

The full NYC taxi dataset won't even fit in the RAM of my laptop. Do I need a cluster yet? No. First, I can take advantage of all the cores on my laptop in parallel. This is what we call *scaling up* our computation (out-of-core computing). Later we'll see how to *scale out* computation across a cluster.

One way of doing this is with [Dask](dask.org/). As we're about to see, part of the value of Dask lies in its API being as close as possible to the PyData APIs we know and love, in this case, Pandas.

In [the words of Matthew Rocklin](https://coiled.io/blog/history-dask/), core developer and co-maintainer of Dask and CEO of Coiled, there was a social goal of Dask:
> Invent nothing. We wanted to be as familiar as possible to what users already knew in the PyData stack

Let's do it!

The plan:

* use Dask to load in **all** of the NYC taxi dataset from 10+ CSVs (8+ GBs), 
* massage the data, 
* engineer a feature,
* compute the average tip as a function of the number of passengers, and
* save to [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) (far more efficient than CSV, but not human-readable).

We'll also dive into the basics of Dask and distributed compute (but we'll execute some code first and dive into this part while it runs!).

In [9]:
# Import Dask parts, spin up a local cluster, and instantiate a Client
from dask.distributed import LocalCluster, Client
cluster = LocalCluster(n_workers=4)
client = Client(cluster)
client

/Users/hugobowne-anderson/opt/anaconda3/envs/data-science-at-scale/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54664 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:54665' processes=4 threads=8, memory=8.59 GB>

In [10]:
%%time

import dask.dataframe as dd

# Import the full dataset (note the Dask API!)
df = dd.read_csv(
    "~/Downloads/data-science-at-scale-master/data_taxi/yellow_tripdata_2019-*.csv", 
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype={'RatecodeID': 'float64',
       'VendorID': 'float64',
       'passenger_count': 'float64',
       'payment_type': 'float64'}

)
df

CPU times: user 234 ms, sys: 69.7 ms, total: 304 ms
Wall time: 355 ms


Dask DataFrame Structure:
                VendorID tpep_pickup_datetime tpep_dropoff_datetime passenger_count trip_distance RatecodeID store_and_fwd_flag PULocationID DOLocationID payment_type fare_amount    extra  mta_tax tip_amount tolls_amount improvement_surcharge total_amount congestion_surcharge
npartitions=117                                                                                                                                                                                                                                                                     
                 float64       datetime64[ns]        datetime64[ns]         float64       float64    float64             object        int64        int64      float64     float64  float64  float64    float64      float64               float64      float64              float64
                     ...                  ...                   ...             ...           ...        ...                ...          ...          ...          ...         ...      ...      ...        ...          ...                   ...          ...                  ...
...                  ...                  ...                   ...             ...           ...        ...                ...          ...          ...          ...         ...      ...      ...        ...          ...                   ...          ...                  ...
                     ...                  ...                   ...             ...           ...        ...                ...          ...          ...          ...         ...      ...      ...        ...          ...                   ...          ...                  ...
                     ...                  ...                   ...             ...           ...        ...                ...          ...          ...          ...         ...      ...      ...        ...          ...                   ...          ...                  ...
Dask Name: read-csv, 117 tasks

In [11]:
# Alter data types for efficiency
df = df.astype({
    "VendorID": "UInt8",
    "passenger_count": "UInt8",
    "RatecodeID": "UInt8",
    "store_and_fwd_flag": "category",
    "PULocationID": "UInt16",
    "DOLocationID": "UInt16",    
})

# Create new feature in dataset: tip_ratio
df["tip_ratio"] = df.tip_amount / df.total_amount

In [12]:
%%time

# Prepare to compute the average tip 
# as a function of the number of passengers
mean_amount = df.groupby("passenger_count").tip_amount.mean()

CPU times: user 29.9 ms, sys: 3.19 ms, total: 33.1 ms
Wall time: 31.1 ms


In [13]:
%%time

# compute the average tip as a function of the number of passengers
mean_amount.compute()

CPU times: user 53.9 s, sys: 7.89 s, total: 1min 1s
Wall time: 5min 14s


passenger_count
0    2.116993
1    2.199312
2    2.206649
3    2.132150
4    2.018263
5    2.227626
6    2.212617
7    6.688656
8    7.107717
9    7.377450
Name: tip_amount, dtype: float64

In [ ]:
%%time
df.to_parquet("data_taxi/yellow_tripdata_2019.parq")

### 2a Notes on what is happening in Dask and Python

The above code will take some time to run so let's take this opportunity to see what is going on with Dask, Python, and the distributed computation.

#### Components of Dask

Dask contains 3 main components and we have already see two of them above:
* High-level collections in the form of Dask DataFrames;
* Schedulers (in this case, on a single machine).

Let's get a sense for what these are.

<img src="images/dask-components.svg" width="400px">

#### Dask DataFrames

What exactly is this Dask DataFrame? A schematic is worth a thousand words:

<img src="images/dask-dataframe.svg" width="400px">

Essentially, the Dask DataFrame is a large, virtual dataframe divided along the index into multiple Pandas dataframes.

#### Dask Schedulers, Workers, and beyond

<img src="images/dask-cluster.svg" width="400px">

Work (Python code) is performed on a cluster, which consists of 
* a _scheduler_ (which manages and sends the work / tasks to the workers)
* _workers_, which compute the tasks.

The _client_ is "the user-facing entry point for cluster users." What this means is that the client lives wherever you are writing your Python code and the client talks to the scheduler, passing it the tasks.

**Recap:** We have

* used Dask to load in **all** of the NYC taxi dataset from 10+ CSVs (8+ GBs), 
* massaged the data, 
* engineered a feature, 
* computed the average tip as a function of the number of passengers, and 
* saved to [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) (far more efficient than CSV, but not human-readable),
* dived into the basic of Dask and distributed compute and understand the basic concepts.

## 3. Optional: Work directly from the cloud with Coiled 

<br>
<img src="images/horizontal.png" alt="Coiled logo" style="width: 500px;"/>
<br>

Here I'll spin up a cluster on Coiled to show you just how easy it can be. Note that to do so, I've also signed into the [Coiled Beta](beta.coiled.io/), pip installed `coiled`, and authenticated. You can do the same!

The plan:

* use Coiled to load in **all** of the NYC taxi dataset from 10+ CSVs (8+ GBs) on an AWS cluster, 
* massage the data, 
* engineer a feature, 
* compute the average tip as a function of the number of passengers, and 
* save to [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) (far more efficient than CSV, but not human-readable).

In [14]:
import coiled
from dask.distributed import LocalCluster, Client

In [ ]:
# Create a Software Environment
coiled.create_software_environment(
    name="my-software-env",
    conda="binder/environment.yml",
)

In [ ]:
# Control the resources of your cluster by creating a new cluster configuration
coiled.create_cluster_configuration(
    name="my-cluster-config",
    worker_memory="16 GiB",
    worker_cpu=4,
    scheduler_memory="8 GiB",
    scheduler_cpu=2,
    software="my-software-env",
)


In [15]:
# Spin up cluster, instantiate a Client
cluster = coiled.Cluster(n_workers=10, configuration="my-cluster-config")
client = Client(cluster)
client

Creating Cluster. This takes about a minute ...Checking environment images
Valid environment image found


distributed.comm.tcp - WARNING - Closing dangling stream in <TLS  local=tls://172.20.10.11:54811 remote=tls://ec2-18-224-150-126.us-east-2.compute.amazonaws.com:8786>
distributed.comm.tcp - WARNING - Closing dangling stream in <TLS  local=tls://172.20.10.11:54813 remote=tls://ec2-18-224-150-126.us-east-2.compute.amazonaws.com:8786>


<Client: 'tls://10.1.12.198:8786' processes=2 threads=8, memory=34.36 GB>

In [16]:
import dask.dataframe as dd

# Read data into a Dask DataFrame
df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv", 
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype={
        'RatecodeID': 'float64',
       'VendorID': 'float64',
       'passenger_count': 'float64',
       'payment_type': 'float64'
    },
    storage_options={"anon":True}
)
df

Dask DataFrame Structure:
                VendorID tpep_pickup_datetime tpep_dropoff_datetime passenger_count trip_distance RatecodeID store_and_fwd_flag PULocationID DOLocationID payment_type fare_amount    extra  mta_tax tip_amount tolls_amount improvement_surcharge total_amount congestion_surcharge
npartitions=127                                                                                                                                                                                                                                                                     
                 float64       datetime64[ns]        datetime64[ns]         float64       float64    float64             object        int64        int64      float64     float64  float64  float64    float64      float64               float64      float64              float64
                     ...                  ...                   ...             ...           ...        ...                ...          ...          ...          ...         ...      ...      ...        ...          ...                   ...          ...                  ...
...                  ...                  ...                   ...             ...           ...        ...                ...          ...          ...          ...         ...      ...      ...        ...          ...                   ...          ...                  ...
                     ...                  ...                   ...             ...           ...        ...                ...          ...          ...          ...         ...      ...      ...        ...          ...                   ...          ...                  ...
                     ...                  ...                   ...             ...           ...        ...                ...          ...          ...          ...         ...      ...      ...        ...          ...                   ...          ...                  ...
Dask Name: read-csv, 127 tasks

In [17]:
%%time

# Prepare to compute the average tip 
# as a function of the number of passengers
mean_amount = df.groupby("passenger_count").tip_amount.mean()

CPU times: user 23.1 ms, sys: 5.72 ms, total: 28.9 ms
Wall time: 29.3 ms


In [18]:
%%time

# Compute the average tip 
# as a function of the number of passengers
mean_amount.compute()

CPU times: user 2.35 s, sys: 359 ms, total: 2.71 s
Wall time: 33.7 s


passenger_count
0.0    2.122789
1.0    2.206790
2.0    2.214306
3.0    2.137775
4.0    2.023804
5.0    2.235441
6.0    2.221105
7.0    6.675962
8.0    7.111625
9.0    7.377822
Name: tip_amount, dtype: float64

In [ ]:
# Alter data types for efficiency
df = df.astype({
    "VendorID": "UInt8",
    "passenger_count": "UInt8",
    "RatecodeID": "UInt8",
    "store_and_fwd_flag": "category",
    "PULocationID": "UInt16",
    "DOLocationID": "UInt16",    
})

# Create new feature in dataset: tip_ratio
df["tip_ratio"] = df.tip_amount / df.total_amount

In [ ]:
%%time
df.to_parquet("s3://hugo-coiled-tutorial/nyctaxi-2019.parq")

In [19]:
cluster.close()

distributed.comm.tcp - WARNING - Closing dangling stream in <TLS  local=tls://172.20.10.11:54843 remote=tls://ec2-18-224-150-126.us-east-2.compute.amazonaws.com:8786>
distributed.comm.tcp - WARNING - Closing dangling stream in <TLS  local=tls://172.20.10.11:54845 remote=tls://ec2-18-224-150-126.us-east-2.compute.amazonaws.com:8786>
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


**Recap:** We have
* used Coiled to load in **all** of the NYC taxi dataset from 10+ CSVs (8+ GBs) on an AWS cluster, 
* massaged the data, 
* engineered a feature, 
* computed the average tip as a function of the number of passengers, and 
* saved to [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) (far more efficient than CSV, but not human-readable).